# Introducing: Feature modifiers

With the current L2 Constructs, all feature abstractions are enclosed within the respective Construct. Take a `Topic` as an example: The L2 includes an abstraction to help with FIFO Topics. It codifies the dependency between some low-level CFN properties and the topic name having to end in `.fifo`. This abstraction is currently *only* available to users of the L2. If I want to use this feature, I *have to* use the L2. This is problematic in scenarios where users are interested in *some*, but not *all* of the features of an L2.

The basic idea behind feature modifiers is to break-out these abstractions from being tightly coupled to an L2. They should be usable with any `Topic`, or - going even further - with any Construct tree that includes `n` `Topics` (with `n >= 1`).

## Basic example: Topic

Taking the aforementioned, the current L2 features can be re-created with these two feature modifiers: `sns.encrypt` to handle encryption and `sns.fifo` to help with FIFO topics.

In [1]:
import './lib/setup.ts';

const topicKey = new kms.Key(stack, "TopicKey");
const sourceTopic = new sns.L1Topic(stack, "IncomingMessages");
sns.encrypt(sourceTopic, topicKey)
sns.fifo(sourceTopic, {
  contentBasedDeduplication: true,
});

print(sourceTopic, topicKey);

TopicKeyB2E0C9CB:
  Type: 'AWS::KMS::Key'
  Properties:
    KeyPolicy:
      Statement:
        - Action: 'kms:*'
          Effect: Allow
          Principal:
            AWS:
              'Fn::Join':
                - ''
                - - 'arn:'
                  - Ref: 'AWS::Partition'
                  - ':iam::'
                  - Ref: 'AWS::AccountId'
                  - ':root'
          Resource: '*'
      Version: '2012-10-17'
  UpdateReplacePolicy: Retain
  DeletionPolicy: Retain
IncomingMessages:
  Type: 'AWS::SNS::Topic'
  Properties:
    kmsMasterKeyId:
      'Fn::GetAtt':
        - TopicKeyB2E0C9CB
        - Arn
    topicName: IncomingMessages.fifo
    fifoTopic: true
    contentBasedDeduplication: true



## Add a Queue as a target for the Topic

To expend our example, we add a `Queue` as a target for a later added `TopicSubscription`. Let's use this new resources to look at possible syntax variations.

### Excursion: Syntax variations

This uses a variation of the modifiers syntax: Instead of directly calling the modifier, we pass it to the `with()` method of the resource construct.s
The downside of it is limited support in code auto-completion. `with()` would accept *any* modifier, so we can't suggest which modifier to use and what parameters the modifier supports. A workaround would be to use overloads in the method definition. With overloads, we get a list of expected modifiers and once the first parameter is provided, the auto-completion appears to be able to pick the right one.

<img src="images/modifiers_with_help.png" width="400" class="blog-image">

Line 8 uses yet another alternative syntax, which improves code auto-completion. These additional methods would have to be codegen'd and would only support built-in modifiers.

<img src="images/modifiers_use_help.png" width="450" class="blog-image">

### Inspired by pipeline operator

The syntax for these is inspired by the *tc39 pipeline operator proposal*. Any variation of it could be considered going forward. Crucially, it is syntactic sugar on top of the base implementation of a function that takes a construct as first parameter. See [this article implementing the pipeline operator proposal](https://dev.to/nexxeln/implementing-the-pipe-operator-in-typescript-30ip) for further information.

In [2]:
const targetQueue = new sqs.L1Queue(stack, "AcceptedMessages")
  .with(sqs.visibilityTimeout, cdk.Duration.seconds(300))
  .with(sqs.enforceSSL)
  .useEncryptWithKey();

print(targetQueue);

AcceptedMessages:
  Type: 'AWS::SQS::Queue'
  Properties:
    visibilityTimeout: 300
    kmsMasterKeyId:
      'Fn::GetAtt':
        - AcceptedMessagesKey1F944073
        - Arn
AcceptedMessagesPolicyA013B031:
  Type: 'AWS::SQS::QueuePolicy'
  Properties:
    PolicyDocument:
      Statement:
        - Action: 'sqs:*'
          Condition:
            Bool:
              'aws:SecureTransport': 'false'
          Effect: Deny
          Principal:
            AWS: '*'
          Resource:
            'Fn::GetAtt':
              - AcceptedMessages
              - Arn
      Version: '2012-10-17'
    Queues:
      - 'Fn::GetAtt':
          - AcceptedMessages
          - QueueUrl
AcceptedMessagesKey1F944073:
  Type: 'AWS::KMS::Key'
  Properties:
    Description: Created by Default/AcceptedMessages
    KeyPolicy:
      Statement:
        - Action: 'kms:*'
          Effect: Allow
          Principal:
            AWS:
              'Fn::Join':
                - ''
                - - 'arn:'
        

### Create a DLQ for the Subscription

`TopicSubscription`s support dead-letter queues for undelivered messages. Let's create an other queue for this purpose, using yet another syntax variation.

### Higher-order programming

So far, the modifier type has been a bit messy. While it generally takes "a thing" as the first parameter, it's wild and undefined what comes after it: Sometimes it's a single value like a `Duration`, at other times we have multiple values (some of which are optional) and even a property bag!

One way to get around this inconsistency, is higher-order programming. With HOP we can strictly define modifiers as taking a single *subject* that executes the modification. Because we still need the ability to pass in values, our "feature modifiers" are now a factory instead: For a given encryption `Key`, make me a feature modifier that encrypts the `Queue` with it.

In [9]:
const dlq = new sqs.L1Queue(stack, "FilteredMessages")
  .with(sqs.encryptWithKeyStrict(new kms.Key(stack, "FilteredQueueKey")));

print(dlq);

FilteredMessages:
  Type: 'AWS::SQS::Queue'
  Properties:
    kmsMasterKeyId:
      'Fn::GetAtt':
        - FilteredQueueKey896A5D37
        - Arn



## Vocabulary

We have already established, that modifiers operate on a *subject*. If we lean into this analogy to the English language grammar, we can get inspiration for further terminology. Note this is not aiming to be an accurate mapping to grammar rules. Natural languages is weird and beautiful and ambagious, whereas code should be clear and unambiguous.

**Subject**

The Construct we are operating with. When multiple Constructs are involved, this is the main one. In a localized Construct tree, it would be the parent. Usually these are CFN Resources or L2s: `Topic`, `Queue` but it can be more complex like an `App`.

**Predicate** or **Action**

The modifier itself. It expresses an action or being. `encrypt()` is an example for action, `fifo()` for being. With this in mind, we could change `visibilityTimeout()` to an arguably more friendly `visibleFor()`.

**Adverb** (⚠️ Not happy with this, modifiers would be better but than the whole concept needs to be renamed)

In natural language, adverbs are used to further modify or describe the predicate. The `contentBasedDeduplication` property is a further description of the `fifo()` being of `Topic`. The duration of `visibleFor()` further modifies how long messages are visible for. Now it becomes more obvious why a higher-order modifier form could be considered as more pure than other variations: Arguments are only used in the context they are relevant for.

**Objects**

Any other Constructs that are part of the action. To `encrypt()` a `Topic` we need a `Key`. So the `Key` becomes the *object*. 

❌ This implementation of `L1Subscription` without receiving a `Topic` and `Queue` at construction time doesn't work. Both `TopicArn` and `Protocol` are required.

In [6]:
const topic2queue = new sub.L1Subscription(stack, "FilterMessages")
  .with(sub.connect(sourceTopic, targetQueue))
  .with(sub.deadLetterQueue(dlq))
  .with(sub.filterMessages({
    color: sns.SubscriptionFilter.stringFilter({
      allowlist: ["red", "orange"],
      matchPrefixes: ["bl"],
    }),
  }));

print(topic2queue, targetQueue.policy);

AcceptedMessagesPolicyA013B031:
  Type: 'AWS::SQS::QueuePolicy'
  Properties:
    PolicyDocument:
      Statement:
        - Action: 'sqs:*'
          Condition:
            Bool:
              'aws:SecureTransport': 'false'
          Effect: Deny
          Principal:
            AWS: '*'
          Resource:
            'Fn::GetAtt':
              - AcceptedMessages
              - Arn
        - Action: 'sqs:SendMessage'
          Condition:
            ArnEquals: {}
          Effect: Allow
          Principal:
            Service: sns.amazonaws.com
          Resource:
            'Fn::GetAtt':
              - AcceptedMessages
              - Arn
      Version: '2012-10-17'
    Queues:
      - 'Fn::GetAtt':
          - AcceptedMessages
          - QueueUrl
FilterMessages:
  Type: 'AWS::SNS::Subscription'
  Properties:
    topicArn:
      'Fn::GetAtt':
        - IncomingMessages
        - TopicArn
    endpoint:
      'Fn::GetAtt':
        - AcceptedMessages
        - Arn
    protocol: s

## Beyond L1s

We now have defined the subject as the Construct we are operating with. But so far all of our subjects have L1 Resources - that's only a subset of Constructs. Taking a step back, why not allow any modifier to be applied to any Construct? There will be some drawbacks in regards to typing, but we can work around this with additional syntax.

The following examples operate on a Stack, but they can work with any Construct tree. Crucially this includes L2- and L3-like Constructs. With this, we now really have broken up the closed nature of current L2s.

In [ ]:
// Apply modifier to any construct tree
sqs.enforceSSL(stack)

// Or maybe this?
const visibleFor500 = any(sqs.L1Queue.isL1Queue, sqs.visibilityTimeoutStrict(cdk.Duration.seconds(500)));
visibleFor500(stack);

// Or maybe something like this:
// any(sqs.L1Queue.isL1Queue).of(stack).must(sqs.enforceSSL);

printOnly(targetQueue);
print(dlq);

## Beyond Constructs

Going even further, the concept can be applied to any DSL that might be used with the CDK. Let's look at the example of a Dashboard definition, consisting of various widgets which each use a Metric. Aside from the `Dashboard` none of these are physical resources. Yet we can apply the same look and feel to them, using modifiers to change a subject. This works by creating "virtual" Constructs in the tree. 

In this concept, **Resources** would be the things that CloudFormation knows about. It's the representation of the AWS Service Model in the CDK. They came in the **physical** and **logical** variety. The former represent reality, and the latter being a desired state. **Virtuals** (naming tbd) are a new addition. These Constructs do not have a representation in the physical or logical world. They only exist as nodes in the construct tree. A consequence of this is, that we need to convert the virtual construct tree into a physical tree at some point. This conversion should happen transparently to the user, since interactions with each of them will be different.

### About polymorphism

With *Virtuals* we suddenly have to deal with multiple different types of Constructs on a tree. You might think. But it's not a completely newly introduced problem. You just have to look at L2s themselves. They are not CFN Resources at all! What they are is a container that holds one or more `CfnElement`, and that is what the framework is looking for when synthesizing your App. Anything that's not implementing the `CfnElement` contract (it uses a Symbol), will be ignored. So the only new thing we are introducing with *Virtuals* is that different subtrees will be interested in different types of Constructs. For a `Dashboard` it would be `Widget`s and `Variable`s, and `Widget`s itself might be interested in `Metric`s.

In [5]:
const graphProcessed = new cw.L1GraphWidget(stack, 'ProcessedGraph')
  .with(cw.title('Processed Messages'))
  .with(cw.onLeft(sourceTopic.metric(sns.metricNumberOfMessagesPublished())))
  .with(cw.onLeft(targetQueue.metric(sqs.metricNumberOfMessagesReceived())));

const graphFiltered = new cw.L1GraphWidget(stack, 'FilteredGraph')
  .with(cw.title('Filtered Messages'))
  .with(cw.onLeft(sourceTopic.metric(sns.metricNumberOfNotificationsFilteredOut())))
  .with(cw.onLeft(dlq.metric(sqs.metricNumberOfMessagesReceived())));

const dashboard = new cw.L1Dashboard(stack, "Dashboard")
  .with(cw.widget(graphProcessed))
  .with(cw.widget(graphFiltered));

print(dashboard);

Dashboard:
  Type: 'AWS::CloudWatch::Dashboard'
  Properties:
    dashboardBody:
      'Fn::Join':
        - ''
        - - >-
            {"widgets":[{"type":"metric","width":6,"height":6,"properties":{"view":"timeSeries","title":"Processed
            Messages","region":"
          - Ref: 'AWS::Region'
          - >-
            ","metrics":[["AWS/SNS","NumberOfMessagesPublished","TopicName","IncomingMessages.fifo",{"stat":"Sum"}],["AWS/SQS","NumberOfMessagesReceived","QueueName","
          - 'Fn::GetAtt':
              - AcceptedMessages
              - QueueName
          - >-
            ",{"stat":"Sum"}]],"yAxis":{}}},{"type":"metric","width":6,"height":6,"properties":{"view":"timeSeries","title":"Filtered
            Messages","region":"
          - Ref: 'AWS::Region'
          - >-
            ","metrics":[["AWS/SNS","NumberOfNotificationsFilteredOut","TopicName","IncomingMessages.fifo",{"stat":"Sum"}],["AWS/SQS","NumberOfMessagesReceived","QueueName","
          - 'Fn::GetAtt

## Everything's a Construct

Once we have a virtual Construct tree, we can lean into it further on put even more things on it. 'What's left?', you ask. The modifiers themselves. There are many different ways we can approach a syntax for this. The main point of contention with this approach is that for a Construct to be a Construct, we need to provide a *scope* and a *identifier*. Imagine this:

```ts
new AddHornModifier(stack, 'AddRainbowHorn', {
  color: 'rainbow'
});
```

It works, but is *very* verbose. Good news is, I believe we can strongly reduce the verbosity of Modifier-Constructs. First of all, if every Modifier operates on a subject then we already have our scope. Modifiers are now attached to the subject they are changing. The second part is the identifier. Let's take a step back and ask ourselves why we even have `id`s? A part of the answer to this is, that CFN templates require unique *Logical Ids* and they must be stable. While are other benefits to having unique Id paths in Construct Tree, this is a fundamental one that cannot be 'engineered away'. With Modifiers (and *Virtuals*) we don't have this constraint. There are fathomable scenarios where stable Construct Ids are useful, but on a whole they are simply not required. Enter `Auto Id`s. We can just generate ids for modifiers (algorithm tbd) and call it a day. Most benefits will remain and complexity goes away.

Here is a possible implementation of this approach.

```ts
let modifierCounter = 1;
export class L1Unicorn extends L1Resource {
  public static paintHorn(hornColor: string) {
    return class extends Construct {
      public constructor(public subject: L1Unicorn) {
        super(subject, `${new.target.name}#${++modifierCounter}`);
        subject.withProperties({ hornColor });
      }
    };
  }
}
```

In [4]:
const forrest = new cdk.Stack();
const unicorn = new magic.L1Unicorn(forrest, 'Bubbles')
  .with(magic.L1Unicorn.paintHorn('rainbow'));

print(unicorn);

Bubbles:
  Type: 'AWS::Magic::Unicorn'
  Properties:
    hornColor: rainbow

